In [3]:
import credentials as cd
from fyers_apiv3 import fyersModel
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import logging

# Configure logging
logging.basicConfig(
    filename="orders_log.txt", level=logging.INFO, format="%(asctime)s - %(message)s"
)

# Read access token
with open("access.txt", "r") as a:
    access_token = a.read()
client_id = cd.client_id

fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=""
)


# Fetch historical data function
def getdata(sym, res, rfrom, rto):
    cdata = {
        "symbol": sym,
        "resolution": str(res),
        "date_format": "1",
        "range_from": rfrom,
        "range_to": rto,
        "cont_flag": "0",
    }

    response = fyers.history(data=cdata)
    data = pd.DataFrame.from_dict(response["candles"])
    cols = ["datetime", "open", "high", "low", "close", "volume"]
    data.columns = cols
    data["datetime"] = pd.to_datetime(data["datetime"], unit="s")
    data["datetime"] = (
        data["datetime"].dt.tz_localize("utc").dt.tz_convert("Asia/Kolkata")
    )
    data["datetime"] = data["datetime"].dt.tz_localize(None)
    data = data.set_index("datetime")
    return data


class EMACrossStrategy(Strategy):
    n1 = 5  # Short EMA period
    n2 = 15  # Long EMA period

    def init(self):
        # Calculate the short and long EMAs
        close = pd.Series(self.data.Close)
        self.ema_short = self.I(
            lambda x: close.ewm(span=self.n1, adjust=False).mean(), "ema_short"
        )
        self.ema_long = self.I(
            lambda x: close.ewm(span=self.n2, adjust=False).mean(), "ema_long"
        )

    def next(self):
        # Check for crossover and trade accordingly
        if crossover(self.ema_short, self.ema_long):
            self.buy()
            logging.info(
                f"Buy - Date: {self.data.index[-1]}, Price: {self.data.Close[-1]}"
            )
        elif crossover(self.ema_long, self.ema_short):
            self.sell()
            logging.info(
                f"Sell - Date: {self.data.index[-1]}, Price: {self.data.Close[-1]}"
            )


# Fetch historical data for backtesting
date_from = "2024-04-03"
date_to = "2024-05-30"
data = getdata("NSE:NIFTYBANK-INDEX", 1, date_from, date_to)

# Prepare data for backtesting
data = data.rename(
    columns={
        "close": "Close",
        "open": "Open",
        "high": "High",
        "low": "Low",
        "volume": "Volume",
    }
)

# Run the backtest
bt = Backtest(
    data, EMACrossStrategy, cash=100000, commission=0.002, exclusive_orders=True
)

output = bt.run()
print(output)
bt.plot()

C:\Users\Shiva Kumar G\AppData\Local\Temp\ipykernel_14080\1090490490.py:94: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(


Start                     2024-04-03 09:15:00
End                       2024-05-17 15:29:00
Duration                     44 days 06:14:00
Exposure Time [%]                    53.96129
Equity Final [$]                    46976.342
Equity Peak [$]                    100198.952
Return [%]                         -53.023658
Buy & Hold Return [%]                1.539937
Return (Ann.) [%]                  -99.717823
Volatility (Ann.) [%]                0.109539
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -53.13849
Avg. Drawdown [%]                  -21.313903
Max. Drawdown Duration       45 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                  514
Win Rate [%]                         5.836576
Best Trade [%]                        0.89322
Worst Trade [%]                     -1.253191
Avg. Trade [%]                    

c:\Users\Shiva Kumar G\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '10T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
c:\Users\Shiva Kumar G\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
c:\Users\Shiva Kumar G\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:128: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  indicators = [_Indicator(i.df.resample(freq, label='right').mean()
c:\Users\Shiva Kumar G\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:128: FutureWarning: 'T

ValueError: Length of values (2) does not match length of index (1)